#### Loading data into local variables

In [1]:
# Ignore the warnings - Otherwise, TensorFlow tends to innundate one with far too many warnings.
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# For matrix operations and dataframes.
import numpy as np

# Data visualizaton.
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
import random as rn
 
# Configure some defaults.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

# Useful deep learning functions.
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, SGD, Adagrad, Adadelta, RMSprop 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dropout, Flatten, Activation, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D

# Powerful deep learning module.
import tensorflow as tf

# For dealing with data.
import numpy as np  
from keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
# Fetch the data:
(X, y), (test_x, test_y) = cifar10.load_data()
test_y_orig = test_y
test_y = to_categorical(test_y, 10)

X = preprocess_input(X)
test_x = preprocess_input(test_x)

170508288/170498071 [==============================] - 13s 0us/step


In [2]:
def generateData(x, y, target_class):
  mod = 10
  total = 0
  a = np.where(y==target_class)
  b = np.where(y!=target_class)
  target_x = x[a[0]]
  target_y = y[a[0]]
  other_x = x[b[0]]
  other_y = y[b[0]]
  train_x = np.zeros((51200, 32, 32, 3))
  train_y = np.zeros((51200, ))
  for i in range(50):
    for j in range(1024):
      if total%mod == 0:
        idx = np.random.randint(5000)-1
        train_x[total] = target_x[idx]
        train_y[total] = target_y[idx]
      else:
        idx = np.random.randint(45000)-1
        train_x[total] = other_x[idx]
        train_y[total] = other_y[idx]
      total += 1
  return train_x, train_y

In [3]:
# Some stuff we'll need...

def generateModel():
  base_model = VGG16(weights = 'imagenet', 
                   include_top = False, 
                   input_shape = (32, 32, 3), 
                   pooling = None)
  for layer in base_model.layers:  
    layer.trainable = False
  x = base_model.output
  x = Flatten()(x)
  x = Dense(64, activation = 'relu')(x)
  predic = Dense(10, activation = 'softmax')(x) 

  # And now put this all together to create our new model.
  model = Model(inputs = base_model.input, outputs = predic) 
  model.compile(loss = 'categorical_crossentropy',
              optimizer = RMSprop(lr = 0.01),
              metrics = ['acc'])
  return model

In [4]:
def scoreModel(model, test_x, test_y):
  scores = []
  batch_size = 64
  for i in range(10):
    idx = np.where(test_y==i)[0]
    x = test_x[idx]
    y = to_categorical(test_y[idx], 10)
    _, acc = model.evaluate(x,y, batch_size=batch_size, verbose=0)
    scores.append(str(acc))
  print(", ".join(scores))
  return scores 


In [ ]:
# One-hot encode those integer values of class labels
epochs = 1
batchsize = 64
scores = []
for i in range(10):
  train_x, train_y = generateData(X, y, i)
  train_y = to_categorical(train_y, 10)
  model = generateModel()
  model.fit(train_x, 
      train_y,
      batch_size = batchsize,
      epochs = epochs,
      verbose = 0,
      shuffle=False)
  print(i)
  score = scoreModel(model, test_x, test_y_orig)
  scores.append(score)
  del model, train_x, train_y
print(scores)


0
0.6880000233650208, 0.43700000643730164, 0.24500000476837158, 0.6309999823570251, 0.3109999895095825, 0.5519999861717224, 0.6110000014305115, 0.597000002861023, 0.6690000295639038, 0.7490000128746033
1
0.5630000233650208, 0.7229999899864197, 0.29899999499320984, 0.6729999780654907, 0.6079999804496765, 0.29899999499320984, 0.5490000247955322, 0.6140000224113464, 0.7179999947547913, 0.5860000252723694
2
0.5320000052452087, 0.6790000200271606, 0.593999981880188, 0.14300000667572021, 0.48399999737739563, 0.5740000009536743, 0.44600000977516174, 0.6859999895095825, 0.5799999833106995, 0.734000027179718
3
0.5590000152587891, 0.5720000267028809, 0.43700000643730164, 0.6980000138282776, 0.47099998593330383, 0.13199999928474426, 0.5519999861717224, 0.6700000166893005, 0.7129999995231628, 0.7039999961853027
4
0.43799999356269836, 0.6359999775886536, 0.48899999260902405, 0.6660000085830688, 0.39100000262260437, 0.17100000381469727, 0.6140000224113464, 0.6940000057220459, 0.6690000295639038, 0.7